In [76]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import importlib
import sklearn
import random
import torch
import math

import socialSigN
importlib.reload(socialSigN)
from helpers import *

In [170]:
import warnings
warnings.filterwarnings('ignore')

In [77]:
####### Load our Data
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'
devSet = pd.read_csv("../us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)
# devSet = devSet.drop(['sending'], axis = 1)

y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values

mMScale = preprocessing.MinMaxScaler()
X = mMScale.fit_transform(X)

In [121]:
xy = devSet[['sending', 'US_MIG_05_10']]
xy.head()

,sending,US_MIG_05_10
0,10001,961
1,10002,154
2,10003,905
3,10004,225
4,10005,1071


In [203]:
sending = devSet['sending'].to_list()
sending[0:5]

[10001, 10002, 10003, 10004, 10005]

In [322]:
####### Build and fit the Model
lr = 1e-5
batchSize = 50
model = socialSigN.SocialSigNet(X=torch.reshape(torch.tensor(read_file(1001), dtype = torch.float32, requires_grad = True), (1, 243)), 
                                outDim = batchSize)
epochs = 1

criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [323]:
def load_data(batchSize, xy, batch, sending_ids):

    batch = [sending_ids[i] for i in batch]
    # print("    Batch ID's: ", batch)
    inputs = [read_file(i) for i in batch]
    inputs = torch.reshape(torch.tensor(inputs, dtype = torch.float32, requires_grad = True), (len(inputs), 243))

    ys = []

    for i in batch:
        # print(i)
        cur = xy[xy['sending'] == int(i)]
        # print(list(cur['US_MIG_05_10'])[0])
        ys.append(list(cur['US_MIG_05_10'])[0])

    ys = torch.reshape(torch.tensor(ys, dtype = torch.float32, requires_grad = True), (len(inputs), 1))

    return inputs, ys

# load_data(5, avail_sending, xy)

In [324]:
def train_test_split(sending_ids, split, batchSize, xy):

    train_num = int(len(sending_ids) * split)
    val_num = int(len(sending_ids) - train_num)

    train_indices = random.sample(range(len(sending_ids)), train_num)
    val_indices = [i for i in range(len(sending_ids)) if i not in train_indices]

    x_train, y_train = load_data(batchSize, xy, train_indices, avail_sending)


    x_val, y_val = load_data(batchSize, xy, val_indices, avail_sending)

    return x_train, y_train, x_val, y_val


x_train, y_train, x_val, y_val = train_test_split(avail_sending, .80, 5, xy)

In [325]:
train = [(k,v) for k,v in zip(x_train, y_train)]
val = [(k,v) for k,v in zip(x_val, y_val)]

train = torch.utils.data.DataLoader(train, batch_size = batchSize, shuffle = True)
val = torch.utils.data.DataLoader(val, batch_size = batchSize, shuffle = True)

In [326]:
def read_file(shape_id):
    fname = "./inputs/" + str(shape_id) + ".0.txt"
    with open(fname, "r") as f:
        f = f.read()

    to_return = []
        
    for i in f.splitlines():
        splt = i.split(" ")
        [to_return.append(float(val)) if val not in ['inf', '-inf', 'nan'] else to_return.append(0) for val in splt]

    return to_return

In [327]:
avail_sending = []
for i in os.listdir("./inputs/"):
    avail_sending.append(i.split(".")[0])

avail_sending = [i for i in avail_sending if i not in ['nan', '']]
avail_sending = [i for i in avail_sending if int(i) in sending]

In [328]:
# def load_data(batchSize, sending_ids, xy):

#     # batch = sending_ids[0:1]
#     batch = random.sample(sending_ids, batchSize)
#     print("    Batch ID's: ", batch)
#     inputs = [read_file(i) for i in batch]
#     inputs = torch.reshape(torch.tensor(inputs, dtype = torch.float32, requires_grad = True), (batchSize, 243))

#     ys = []

#     for i in batch:
#         # print(i)
#         cur = xy[xy['sending'] == int(i)]
#         # print(list(cur['US_MIG_05_10'])[0])
#         ys.append(list(cur['US_MIG_05_10'])[0])

#     ys = torch.reshape(torch.tensor(ys, dtype = torch.float32, requires_grad = True), (batchSize, 1))

#     return inputs, ys

# # load_data(5, avail_sending, xy)

In [316]:
best_mae = 9000000000000000000
best_model_wts = deepcopy(model.state_dict())



for epoch in range(2):

    for phase in ['train','val']:

        if phase == 'train':

            c = 1
            running_train_mae, running_train_loss = 0, 0

            for inputs, output in train:

                if len(inputs) == batchSize:

                    c += 1

                    # print("Epoch: ", epoch)

                    # Forward pass
                    y_pred = model(inputs, str(epoch))
                    loss = criterion(y_pred, output)  

                    # print("    Loss: ", loss)
                    # print("    MAE: ", mae(y_pred, output).item())
                    # print("    Predictions: ", y_pred)
                    # print("    True: ", output)

                    # Zero gradients, perform a backward pass, and update the weights.
                    optimizer.zero_grad()
                    grad = torch.autograd.grad(outputs = loss, inputs = inputs, retain_graph = True)
                    # print(grad)
                    loss.backward()
                    optimizer.step()

                    # Update the coordinate weights
                    # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
                    with torch.no_grad():
                        for name, p in model.named_parameters():
                            if name == 'SocialSig.W':
                                new_val = update_function(p, grad[0], loss, lr)
                                p.copy_(new_val)


                    running_train_mae += mae(y_pred, output).item()
                    running_train_loss += loss.item()

        if phase == 'val':

                c = 1
                running_val_mae, running_val_loss,  = 0, 0

                # print("In validation")

                for inputs, output in val:

                    if len(inputs) == batchSize:

                        # Forward pass
                        y_pred = model(inputs, 4444)
                        loss = criterion(y_pred, output)  

                        running_val_mae += mae(y_pred, output).item()
                        running_val_loss += loss.item()
                        
                        if mae(y_pred, output).item() < best_mae:
                            best_mae = mae(y_pred, output).item()
                            best_model_wts = deepcopy(model.state_dict())


    print("\n")

    print("Epoch: ", epoch)  
    print("  Train:")
    print("    Loss: ", running_train_loss / c)      
    print("    MAE: ", running_train_mae / c)
    print("  Val:")
    print("    Loss: ", running_val_loss / c)      
    print("    MAE: ", running_val_mae / c)

KeyboardInterrupt: 